## Imagine that we have height vs age data for people fed diet a vs b

In [1]:
import numpy as np
from scipy import stats

### First diet a

In [2]:

height = np.array([12,11,13,17,24,22,25,33,31])
age = np.array([1,1,1,2,2,2,3,3,3])

z, c = np.polyfit(age, height, 1, cov=True)
slope_diet_a = z[0]
intercept_diet_a = z[1]
stdev_slope_diet_a = np.sqrt(c[0][0])
stdev_intercept_diet_a = np.sqrt(c[1][1])
print("slope = %.2f +/- %.2f"%(slope_diet_a,stdev_slope_diet_a ))
print("intercept = %.2f +/- %.2f"%(intercept_diet_a, stdev_intercept_diet_a ))

slope = 8.83 +/- 1.45
intercept = 3.22 +/- 3.12


### Then diet b

In [3]:

height = np.array([15,17,14,27,24,29,35,33,39])
age = np.array([1,1,1,2,2,2,3,3,3])

z, c = np.polyfit(age, height, 1, cov=True)
slope_diet_b = z[0]
intercept_diet_b = z[1]
stdev_slope_diet_b = np.sqrt(c[0][0])
stdev_intercept_diet_b = np.sqrt(c[1][1])
print("slope = %.2f +/- %.2f"%(slope_diet_b,stdev_slope_diet_b ))
print("intercept = %.2f +/- %.2f"%(intercept_diet_b, stdev_intercept_diet_b ))

slope = 10.17 +/- 1.14
intercept = 5.56 +/- 2.45


### Ratio in growth rates

In [4]:
ratio = slope_diet_a / slope_diet_b
stdev_ratio = ratio * np.sqrt( (stdev_slope_diet_a / slope_diet_a)**2 + (stdev_slope_diet_b/slope_diet_b)**2 )
print("ratio = %.2f +/- %.2f"%(ratio,stdev_ratio))

ratio = 0.87 +/- 0.17


### Calculate a linear least-squares regression for two sets of measurements
* ### p-value that you can reject hypothesis that slope is zero
* ### correlation coefficient

In [7]:
slope, intercept, r_value, p_value, std_err = stats.linregress(age, height)
print("p-value = %.2E"%p_value)
print("r_value = %.2f"%r_value)

p-value = 1.47E-05
r_value = 0.97
